In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
from keras.utils import np_utils
import pandas as pd
import numpy as np
from google.cloud import storage
import matplotlib.pyplot as plt
import math

%matplotlib inline
np.random.seed(5)

In [ ]:
# 손실 이력 클래스 정의

class LossHistory(keras.callbacks.Callback):
    def init(self):
        self.losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
# 파일 로드
test=pd.read_csv('bike_demand_test.csv')
train=pd.read_csv('bike_demand_train.csv')

In [ ]:
# 데이터셋
x_train = np.array(train.loc[:,'bef_1':'bef_4'])
y_train = np.array(train['rent_freq'])

x_test = np.array(test.loc[:,'bef_1':'bef_4'])
y_test = test['rent_freq']

x_train = np.reshape(x_train, (len(x_train), 4, 1))
x_test = np.reshape(x_test, (len(x_test), 4, 1))

In [ ]:
# 모델구성

model = Sequential()
model.add(GRU(256, batch_input_shape = (360, 4, 1),stateful = True, return_sequences= False)) 
model.add(Dense(1))

In [ ]:
# 모델학습과정설정

adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam)
model.summary() # None,None,1270이나와야함

In [ ]:
# 모델학습

num_epochs = 100
history = LossHistory() 
history.init()
for epoch_idx in range(num_epochs):
    print ('epochs : ' + str(epoch_idx) )
    hist = model.fit(x_train, y_train, epochs=1, batch_size=360, verbose=1, shuffle=False, callbacks=[history])
    model.reset_states()

In [ ]:
# 모델학습과정 살펴보기

plt.plot(history.losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
# 모델 평가하기

scores = model.evaluate(x_test, y_test, batch_size=360)
print('Test Score: ', math.sqrt(scores))

In [ ]:
# 예측값 저장

pred_out = model.predict(x_test, batch_size=360)
csv_out=pd.DataFrame(pred_out)
csv_out.to_csv('gru_25_360.csv')

In [ ]:
#모델저장

from keras.models import load_model
model.save('gru_25_360_1_78.h5') 